In [24]:
library(dplyr)
library(reshape2)
library(knitr)
library(tidyr)


Attaching package: ‘tidyr’

The following object is masked from ‘package:reshape2’:

    smiths



## Homework 2

### 1. Generate a tree graph that represents flipping a coin 4 times,  let A be the event “the first outcome is tails” , B the event “ the second outcome is head” and C the event “the third outcome is tails” calculate 

p(AUBUC) = P (A U B U C) = P(A) + P(B) + P(C) - P(A ∩ B) - P(A ∩ C) - P(B ∩ C) + P(A ∩ B ∩ C)

**Note**: Couldn't get `igraph` to work on JupyterHub. Did in R Studio instead.

In [ ]:
#p(AUBUC) = P (A U B U C) = P(A) + P(B) + P(C) - P(A ∩ B) - P(A ∩ C) - P(B ∩ C) + P(A ∩ B ∩ C)
prob <- .5 + 0.5 + 0.5 - 0.5**2 - 0.5** 2 - 0.5**2 - 0.5**3
prob

![Decision Tree](./DecisionTree.png)

In [ ]:
g <- graph.tree(n = 2^5 - 1, children = 2) 

n_l = c("H","T")
node_labels <- c("",replicate(15,n_l))

edge_labels <- c("1/2")
edge_label2 = replicate(30,edge_labels)


V(g)$color <- "#8bc4a9"

coords <- layout_(g, as_tree())
coord2 = matrix(c(-coords[,2],-coords[,1]),ncol = 2)

plot(g,
     layout = coord2,           # draw graph as tree
     vertex.size = 20,                  # node size
     vertex.color = V(g)$color,          # node color
     vertex.label = node_labels,        # node labels
     vertex.label.cex = 1,             # node label size
     vertex.label.family = "Helvetica", # node label family
     vertex.label.font = 2,             # node label type (bold)
     vertex.label.color = '#000000',    # node label size
     edge.label = edge_label2,          # edge labels
     edge.label.cex = .7,               # edge label size
     edge.label.family = "Helvetica",   # edge label family
     edge.label.font = 1,               # edge label font type (bold)
     edge.label.color = '#000000',      # edge label color
     edge.arrow.size = 0.2,              # arrow size
     edge.arrow.width = 1              # arrow width
)

### 2. From the Dataset Diabetes, construct contingency tables for the following variable combinations:

A: location Vs gender
B: Gender Vs frame
C: Gender Vs Age (Convert age to an discrete ordinal variable with three categories)
D: Cholesterol Vs Age (Convert age and cholesterol to an discrete ordinal variable with three categories)

calculate the joint and marginal probabilities, and from the above contingency tables choose 5 conditional probability examples with the probabilities calculations and one or two sentences explaining the results.

In [2]:
df <- read.csv(file='diabetes.csv')
head(df)

id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn
1000,203,82,56,3.6,4.31,Buckingham,46,female,62,121,medium,118,59,NA,NA,29,38,720
1001,165,97,24,6.9,4.44,Buckingham,29,female,64,218,large,112,68,NA,NA,46,48,360
1002,228,92,37,6.2,4.64,Buckingham,58,female,61,256,large,190,92,185,92,49,57,180
1003,78,93,12,6.5,4.63,Buckingham,67,male,67,119,large,110,50,NA,NA,33,38,480
1005,249,90,28,8.9,7.72,Buckingham,64,male,68,183,medium,138,80,NA,NA,44,41,300
1008,248,94,69,3.6,4.81,Buckingham,34,male,71,190,large,132,86,NA,NA,36,42,195


In [63]:
# Helper functions

# grouped_df - grouped by two variables
group_df_by_vars = function(x, y, df) {
    grouped <- df %>%
    group_by_at(.vars=c(x, y)) %>%
    summarize(n=n())
    
    return(grouped)

}

# con_table - contingency table
create_con_table = function(x, y, grouped_df) {
    #df <- df %>% 
        #group_by_at(.vars=c(x, y)) %>%
        #summarize(n=n())
    
    con_table <- 
        grouped_df %>%
        ungroup() %>%
        dcast(paste(x, "~", y), value.nar = "n")
        
    
    return(con_table)
        
}


# prop_df - grouped by two variables, turned into proportions
create_prop_df = function(grouped_df) {
    prop_df <- 
        grouped_df %>%
        ungroup() %>%
        mutate(prop = n/sum(n))
    
    return(prop_df)
}





# joint_df - proportions dcast, joint probabilities
compute_joint_probs = function(x, y, prop_df) {
    joint_df <-
        prop_df %>%
        #ungroup() %>%
        #mutate(prop = n / sum(n)) %>% 
        dcast(paste(x, "~", y), value.var = "prop")
    return (joint_df)
}



# Now compute the marginal probs for x and y
# x_marginal_df - marginal probs for x

compute_marg_probs_one_var = function(col, prop_df) {
    marg_df <- prop_df %>%
    group_by_at(.vars=c(col)) %>%
    #group_by(col)
    summarize(marginal = sum(prop))
}

#x_marg_df = compute_marg_probs_one_var("location", prop_df)


# Now join them
# marginal_df - marginal tables joined together
#x = "location"
#y = "gender"

compute_marginal_probs = function(x, y, prop_df) {
    x_marg_df <- compute_marg_probs_one_var(x, prop_df)
    y_marg_df = compute_marg_probs_one_var(y, prop_df)
    marginal_df <- prop_df %>%
        dcast(paste(x, "~", y), value.var="prop") %>%
        left_join(x_marg_df, by=x) %>%
        bind_rows(
            y_marg_df %>%
                mutate(!!x := "marginal") %>%
                #dcast(location ~ gender, value.var = "marginal")
                dcast(paste(x, "~", y), value.var = "marginal")
        )
}

## A: Location vs. gender

In [58]:
# First, group by the variables
# Then compute the proportions
grouped_df = group_df_by_vars('location', 'gender', df)
prop_df = create_prop_df(grouped_df)
prop_df

location,gender,n,prop
Buckingham,female,114,0.2828784
Buckingham,male,86,0.2133995
Louisa,female,120,0.2977667
Louisa,male,83,0.2059553


In [54]:
# Joint probabilities
joint_df <- compute_joint_probs("location", "gender", prop_df)
joint_df

location,female,male
Buckingham,0.2828784,0.2133995
Louisa,0.2977667,0.2059553


In [55]:
# Marginal probabilities
marginal_df <- compute_marginal_probs("location", "gender", prop_df)
marginal_df

Warning message in bind_rows_(x, .id):
“binding factor and character vector, coercing into character vector”Warning message in bind_rows_(x, .id):
“binding character and factor vector, coercing into character vector”

location,female,male,marginal
Buckingham,0.2828784,0.2133995,0.4962779
Louisa,0.2977667,0.2059553,0.5037221
marginal,0.5806452,0.4193548,NA


### Conditional Probabilities A
1. 
$$P(female|Buckingham) = \frac{P(female, Buckingham)}{P(Buckingham)} = 0.28/0.5 = 0.57 $$

57% of Buckingham residents are female. This accounts for a total of 28% of the sample.

In [60]:
# Conditional Probabilities
# Probability of female given Buckingham
(prop_df %>%
    filter(gender == "female", location == "Buckingham") %>%
    .$prop ) /
(marginal_df %>%
    filter(location == "Buckingham") %>%
    .$marginal
)


[1] 0.57

## B: Gender Vs frame 

In [18]:
grouped_df <- group_df_by_vars('gender', 'frame', df)
prop_df <- create_prop_df(grouped_df)
prop_df <- prop_df %>%
    filter(frame != "")
prop_df

gender,frame,n,prop
female,large,42,0.10421836
female,medium,116,0.28784119
female,small,69,0.17121588
male,large,61,0.15136476
male,medium,68,0.16873449
male,small,35,0.08684864


In [19]:
# Joint probabilities
joint_df <- compute_joint_probs("gender", "frame", prop_df)
joint_df

gender,large,medium,small
female,0.1042184,0.2878412,0.17121588
male,0.1513648,0.1687345,0.08684864


In [20]:
# Marginal probabilities
marginal_df <- compute_marginal_probs("gender", "frame", prop_df)
marginal_df

Warning message in bind_rows_(x, .id):
“binding factor and character vector, coercing into character vector”Warning message in bind_rows_(x, .id):
“binding character and factor vector, coercing into character vector”

gender,large,medium,small,marginal
female,0.1042184,0.2878412,0.17121588,0.5632754
male,0.1513648,0.1687345,0.08684864,0.4069479
marginal,0.2555831,0.4565757,0.25806452,NA


### Conditional Probabilities B

2.$$P(male|small) = \frac{P(male, small)}{P(small)} = 0.09/0.26 = 0.34 $$

If a sample person is known to be small, there is approximately a 34% chance that they are also a male.

In [ ]:
# Conditional probabilities


## C: Gender Vs Age 
(Convert age to an discrete ordinal variable with three categories) 

In [21]:
# First convert to discrete variable
bin_size = round((max(df$age) - min(df$age) )/ 3)

[1] 24

In [23]:
df$age_cat <- cut(df$age, seq(min(df$age), max(df$age), bin_size),
                    right=FALSE, labels=c(1:3))
unique(df$age_cat)

[1] 2    1    3    <NA>
Levels: 1 2 3

In [25]:
# Filter out rows with NA
df_c <- df %>%
    drop_na(age_cat)
grouped_df = group_df_by_vars('gender', 'age_cat', df_c)
prop_df = create_prop_df(grouped_df)


In [26]:
# Joint probabilities
joint_df <- compute_joint_probs('gender', 'age_cat', prop_df)
joint_df

gender,1,2,3
female,0.2768080,0.2418953,0.05985037
male,0.1720698,0.1870324,0.06234414


In [28]:
# Marginal probabilities
marginal_df <- compute_marginal_probs("gender", "age_cat", prop_df)
marginal_df

Warning message in bind_rows_(x, .id):
“binding factor and character vector, coercing into character vector”Warning message in bind_rows_(x, .id):
“binding character and factor vector, coercing into character vector”

gender,1,2,3,marginal
female,0.2768080,0.2418953,0.05985037,0.5785536
male,0.1720698,0.1870324,0.06234414,0.4214464
marginal,0.4488778,0.4289277,0.12219451,NA


### Conditional Probabilities C

3.$$P(Female | Age Category 3) = \frac{P(Age Category 3, Female)}{P(Age Category 3)} = 0.06/0.12 = 0.49 $$

Approximately 49% of the oldest age category members are female.

4.$$P(Age Category 1 | Male) = \frac{P(Age Category 1, Male)}{P(Male)} = 0.17/0.42 = 0.41 $$

Approximately 41% of males are in the youngest age category.

## D: Cholesterol Vs Age (Convert age and cholesterol to an discrete ordinal variable with three categories)

In [40]:
df_d <- df %>% 
    drop_na(chol, age_cat)

In [44]:
bin_size = round((max(df_d$chol) - min(df_d$chol) )/ 3) - 1
#bin_size

[1] 121

In [46]:
df_d$chol_cat <- cut(df_d$chol, seq(min(df_d$chol), max(df_d$chol), bin_size),
                    right=FALSE, labels=c(1:3))
head(df_d)

id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,⋯,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,age_cat,chol_cat
1000,203,82,56,3.6,4.31,Buckingham,46,female,62,⋯,medium,118,59,NA,NA,29,38,720,2,2
1001,165,97,24,6.9,4.44,Buckingham,29,female,64,⋯,large,112,68,NA,NA,46,48,360,1,1
1002,228,92,37,6.2,4.64,Buckingham,58,female,61,⋯,large,190,92,185,92,49,57,180,2,2
1003,78,93,12,6.5,4.63,Buckingham,67,male,67,⋯,large,110,50,NA,NA,33,38,480,3,1
1005,249,90,28,8.9,7.72,Buckingham,64,male,68,⋯,medium,138,80,NA,NA,44,41,300,2,2
1008,248,94,69,3.6,4.81,Buckingham,34,male,71,⋯,large,132,86,NA,NA,36,42,195,1,2


In [49]:
df_d <- df_d %>%
    drop_na(chol_cat, age_cat)

In [50]:
grouped_df = group_df_by_vars('chol_cat', 'age_cat', df_d)
prop_df = create_prop_df(grouped_df)

In [51]:
# Joint probabilities
joint_df <- compute_joint_probs('chol_cat', 'age_cat', prop_df)
joint_df

chol_cat,1,2,3
1,0.250626566,0.162907268,0.03007519
2,0.195488722,0.255639098,0.09273183
3,0.005012531,0.007518797,NA


In [52]:
# Marginal probabilities
marginal_df <- compute_marginal_probs('chol_cat', 'age_cat', prop_df)
marginal_df

Warning message in bind_rows_(x, .id):
“binding factor and character vector, coercing into character vector”Warning message in bind_rows_(x, .id):
“binding character and factor vector, coercing into character vector”

chol_cat,1,2,3,marginal
1,0.250626566,0.162907268,0.03007519,0.44360902
2,0.195488722,0.255639098,0.09273183,0.54385965
3,0.005012531,0.007518797,NA,0.01253133
marginal,0.451127820,0.426065163,0.12280702,NA


### Conditional Probabilities C

5.$$P(YoungestAgeCategory | HighestCholesterolCategory) = \frac{P(Age Category 1, CholesterolCategory3)}{P(CholesterolCategory3)} = 0.03/0.12 = 0.24 $$

Given that a sample person is in the highest range of cholesterol, there is only a 24% chance that they are also in the youngest age category.

### 3. Baye’s Rule

Write a function in R that allows you to use the Baye’s rule for multiple events.

Use it to calculate the following problem:

Different isoforms of fundamental Hormones such as testosterone are relatively associated to behavioral differences in humans such as extreme aggression.  

In a study where **75% of the participants had the Isoform A**, and **25% had the isoform B.**:
- 54 people exhibited extreme aggression out 95  that had Isoform A, and 
- 34 people with extreme aggression out of 90 that had the isoform B. 

Calculate the probability that someone with the isoform A exhibits extreme aggression. 
Calculate the probability that someone with the isoform B exhibits extreme aggression. 

In [ ]:
data()

In [ ]:
# P(Isoform A) = 0.75
# P(Isoform B) = 0.25
# P(Aggression | Isoform A) = 54/95
# P(Aggression | Isoform B) = 34/90

# P(Aggression | Isoform A)

In [ ]:
# B - subject has extreme aggression
# A_1 - subject had Isoform A
# A_2 - subject had Isofrm B

# Problem 1:
# P(Extreme Agression | Isoform A) = P(B_)

We can generalize the Baye's rule to multiple events such as

$$ P(A_i \mid B) = \frac{P(B \mid A_i) \, P(A_i)}{P(B \mid A_i) P(A_i)+...+P(B \mid A_n)(P(A_n} $$


In [ ]:
bayes_rule = function()